# Fit the Model to Training Data

## Setup

In [2]:
from importlib import reload

import numpy as np
import tensorflow as tf

from hot_dust import preprocess, model

2024-01-25 22:25:09.497593: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-25 22:25:09.537803: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-25 22:25:09.537838: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-25 22:25:09.538493: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-25 22:25:09.543828: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-25 22:25:09.544357: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [23]:
# "reload" to get changes in preprocess.py without restarting the kernel
reload(preprocess)
reload(model)
from hot_dust.preprocess import prepare_training_data, split_training_data
from hot_dust.model import to_tensorflow, compile, pretraining

### Parameters

In [9]:
epochs = 500  # Max 500
batch_size = 64
buffer_size = 10 * batch_size

ds = prepare_training_data()
train, validate, test = to_tensorflow(split_training_data(ds))

train = train.shuffle(buffer_size).batch(batch_size)
validate = validate.batch(batch_size).cache()

In [4]:
layer = pretraining(dataset=test)
network = compile(normalization=layer)
network.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 9)]               0         
                                                                 
 normalization (Normalizati  (None, 9)                 19        
 on)                                                             
                                                                 
 dense (Dense)               (None, 16)                160       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 468 (1.83 KB)
Trainable params: 449 (1.75 KB)
Non-trainable params: 19 (80.00 Byte)
_____________________________

### Fitting

In [5]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=20,
    verbose=1,
)
# List for callbacks
callbacks_list = [early_stopping]

fit = network.fit(
    train,
    epochs=epochs,
    validation_data=validate,
    verbose=2,
    callbacks=[early_stopping],
)

network.save("data/network")
fit = {"epoch": fit.epoch, **fit.history}
np.savez("data/fit.npz", **fit)

Epoch 1/500
3122/3122 - 4s - loss: 0.5005 - val_loss: 0.3458 - 4s/epoch - 1ms/step
Epoch 2/500
3122/3122 - 3s - loss: 0.3036 - val_loss: 0.2761 - 3s/epoch - 1ms/step
Epoch 3/500
3122/3122 - 3s - loss: 0.2537 - val_loss: 0.2365 - 3s/epoch - 1ms/step
Epoch 4/500
3122/3122 - 3s - loss: 0.2200 - val_loss: 0.2059 - 3s/epoch - 1ms/step
Epoch 5/500
3122/3122 - 3s - loss: 0.1936 - val_loss: 0.1858 - 3s/epoch - 1ms/step
Epoch 6/500
3122/3122 - 3s - loss: 0.1750 - val_loss: 0.1683 - 3s/epoch - 1ms/step
Epoch 7/500
3122/3122 - 3s - loss: 0.1600 - val_loss: 0.1555 - 3s/epoch - 1ms/step
Epoch 8/500
3122/3122 - 3s - loss: 0.1485 - val_loss: 0.1445 - 3s/epoch - 1ms/step
Epoch 9/500
3122/3122 - 3s - loss: 0.1387 - val_loss: 0.1351 - 3s/epoch - 1ms/step
Epoch 10/500
3122/3122 - 3s - loss: 0.1312 - val_loss: 0.1290 - 3s/epoch - 1ms/step
Epoch 11/500
3122/3122 - 3s - loss: 0.1250 - val_loss: 0.1233 - 3s/epoch - 1ms/step
Epoch 12/500
3122/3122 - 3s - loss: 0.1199 - val_loss: 0.1181 - 3s/epoch - 1ms/step
E

INFO:tensorflow:Assets written to: data/network/assets


### Training and Validation Losses

In [51]:
fit = np.load("data/fit.npz")

In [52]:
model.plot_loss(fit)

:NdOverlay   [split]
   :Scatter   [epoch]   (value)